In [82]:
import pandas as pd

### 전체 역내 상가 갯수 / 상가 면적 / 공실 면적 을 알기 위한 코드 작성

#### 상가 정보에 대한 데이터 로드

In [ ]:
rental = pd.read_csv('../data/부산교통공사_임대시설물 현황(2022년).csv',encoding='cp949')
rental.head()

,호선,역사명,상가명,업종,면적,수량,계약시작일,계약종료일
0,1호선,다대포해수욕장,CU,브랜드편의점,53,1,2017-04-20,2022-04-19
1,1호선,다대포해수욕장,상가2호,상가,15,1,2019-12-03,2024-12-02
2,1호선,다대포항,CU,브랜드편의점,76,1,2017-04-20,2022-04-19
3,1호선,낫개,CU,브랜드편의점,24,1,2017-04-20,2022-04-19
4,1호선,신장림,CU,브랜드편의점,78,1,2017-04-20,2022-04-19


#### 미남(3) 미남(4) 와 같은 형식의 역이름을 미남 으로 변경

In [84]:
for idx,name in enumerate(rental.역사명):
    if '(' in name:
        rental.역사명.loc[idx] = name.split('(')[0]

rental[rental.역사명=='미남']

C:\Users\YU\AppData\Local\Temp\ipykernel_27872\1107810427.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rental.역사명.loc[idx] = name.split('(')[0]


,호선,역사명,상가명,업종,면적,수량,계약시작일,계약종료일
352,3호선,미남,상가1호,상가,16,1,2020-04-27,2025-04-26
359,4호선,미남,상가2-1호,상가,57,1,2019-10-28,2021-10-27
360,4호선,미남,상가2-2호,상가,57,1,2019-10-28,2021-10-27
361,4호선,미남,"상가 1, 2-3, 3, 4, 5, 6, 7, 8",상가,822,8,2021-01-01,2022-12-31


In [85]:
rental.loc[370:372]

,호선,역사명,상가명,업종,면적,수량,계약시작일,계약종료일
370,4호선,반여농산물시장,상가1호,상가,42,1,2017-09-11,2022-09-10
371,4호선,석대,상가1호,상가,10,1,NaN,NaN
372,4호선,영산대,상가1호,상가,9,1,2019-03-19,2024-03-18


#### 상가가 있는 역을 뽑고 공실은 계약 시작일을 0으로 만들어서 공실 여부 판단

In [86]:
station_name = list(set(rental.역사명))
len(station_name)

97

In [87]:
rental.fillna(0,inplace=True)
rental.tail()

,호선,역사명,상가명,업종,면적,수량,계약시작일,계약종료일
370,4호선,반여농산물시장,상가1호,상가,42,1,2017-09-11,2022-09-10
371,4호선,석대,상가1호,상가,10,1,0,0
372,4호선,영산대,상가1호,상가,9,1,2019-03-19,2024-03-18
373,4호선,동부산대학,상가1호,상가,18,1,2020-03-01,2022-02-28
374,4호선,안평,매점,매점,10,1,2021-06-11,2026-06-10


#### 상가 정보를 뽑아서 저장 할 딕셔너리(map 리스트) 지정

- store_dic : { '역명':['총 상가 수','공실 상가 수','전체 상가 면적','공실 상가 면적'] }

In [88]:
# ['1호선', '다대포해수욕장', 'CU', '브랜드편의점', 53, 1, '2017-04-20', '2022-04-19']

store_dic = {}

for lines in rental.iloc:
    line = list(lines)
    if line[3]=='주차장':
        continue
    if line[1] in store_dic:
        store_dic[line[1]][0] += line[5]
        store_dic[line[1]][2] += line[4]
        if not line[6]:
            store_dic[line[1]][1] += line[5]
            store_dic[line[1]][3] += line[4]
    else:
        store_dic[line[1]] = [0]*4
        store_dic[line[1]][0] += line[5]
        store_dic[line[1]][2] += line[4]
        if not line[6]:
            store_dic[line[1]][1] += line[5]
            store_dic[line[1]][3] += line[4]

In [89]:
print(len(store_dic.keys()))

97


In [90]:
station_store_info = pd.DataFrame({'역사명':store_dic.keys()})
station_store_info.head()

,역사명
0,다대포해수욕장
1,다대포항
2,낫개
3,신장림
4,장림


In [91]:
station_store_info['총 상가 수'] = None
station_store_info['공실 상가 수'] = None
station_store_info['전체 상가 면적'] = None
station_store_info['공실 상가 면적'] = None

station_store_info.head()

,역사명,총 상가 수,공실 상가 수,전체 상가 면적,공실 상가 면적
0,다대포해수욕장,None,None,None,None
1,다대포항,None,None,None,None
2,낫개,None,None,None,None
3,신장림,None,None,None,None
4,장림,None,None,None,None


In [92]:
# [2, 0, 68, 0]

for idx,line in enumerate(store_dic.values()):
    station_store_info['총 상가 수'].iloc[idx] = line[0]
    station_store_info['공실 상가 수'].iloc[idx] = line[1]
    station_store_info['전체 상가 면적'].iloc[idx] = line[2]
    station_store_info['공실 상가 면적'].iloc[idx] = line[3]

station_store_info.head()

,역사명,총 상가 수,공실 상가 수,전체 상가 면적,공실 상가 면적
0,다대포해수욕장,2,0,68,0
1,다대포항,1,0,76,0
2,낫개,1,0,24,0
3,신장림,1,0,78,0
4,장림,1,0,24,0


In [93]:
station_store_info.to_csv('../data/total_data/지하철_상가_임대_정보.csv',encoding='cp949')